# Image Classification: Cats vs Dogs

This notebook tackles binary image classification using both a classical machine learning baseline and a modern transfer learning approach to distinguish cats from dogs.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)

DATA_DIR = 'data/'
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train_images')
TEST_IMG_DIR = os.path.join(DATA_DIR, 'test_images')
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 5

## 1 -- Data Overview

In [2]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
train_df['filename'] = train_df['id'].astype(str) + '.jpg'

plt.figure(figsize=(5, 3.5))
sns.countplot(data=train_df, x='label', palette='viridis')
plt.title('Training Set: Cats vs Dogs')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig('cv_dist_v2.png', dpi=150)
plt.show()

The dataset is evenly balanced with ~5,000 images per class.

## 2 -- Train / Validation Split

In [3]:
train_data, val_data = train_test_split(
    train_df, test_size=0.3, random_state=100, stratify=train_df['label']
)
print(f"Train: {len(train_data)}  |  Validation: {len(val_data)}")

Train: 7000  |  Validation: 3000


## 3 -- Classical Baseline: Random Forest on Colour Histograms
Rather than gradient-based features (HOG), we extract HSV colour histograms (32 bins per channel = 96 features). Colour distribution provides a lightweight yet informative descriptor for natural images.

In [4]:
HIST_SIZE = (64, 64)

def extract_color_hist(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, HIST_SIZE)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    features = []
    for ch in range(3):
        hist = cv2.calcHist([hsv], [ch], None, [32], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.extend(hist)
    return np.array(features)

print("Extracting colour histograms...")
train_hist = np.array([extract_color_hist(os.path.join(TRAIN_IMG_DIR, f)) for f in train_data['filename'].values])
val_hist   = np.array([extract_color_hist(os.path.join(TRAIN_IMG_DIR, f)) for f in val_data['filename'].values])

label_map = {'cat': 0, 'dog': 1}
y_train_rf = train_data['label'].map(label_map).values
y_val_rf   = val_data['label'].map(label_map).values

print(f"Feature vector length: {train_hist.shape[1]}")

rf = RandomForestClassifier(n_estimators=200, random_state=100, n_jobs=-1)
rf.fit(train_hist, y_train_rf)
rf_preds = rf.predict(val_hist)
rf_acc = accuracy_score(y_val_rf, rf_preds)

print(f"\nRandom Forest Accuracy: {rf_acc:.4f}")
print(classification_report(y_val_rf, rf_preds, target_names=['cat', 'dog']))

Extracting colour histograms...


Feature vector length: 96



Random Forest Accuracy: 0.6367
              precision    recall  f1-score   support

         cat       0.63      0.67      0.65      1497
         dog       0.65      0.61      0.63      1503

    accuracy                           0.64      3000
   macro avg       0.64      0.64      0.64      3000
weighted avg       0.64      0.64      0.64      3000



The colour-histogram baseline gives a reasonable starting point. We now leverage deep features via transfer learning to improve substantially.

## 4 -- Transfer Learning with MobileNetV2
MobileNetV2, pretrained on ImageNet, already encodes rich visual features. We freeze the convolutional base and train only a lightweight classification head on top. Images are resized to 150x150.

In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
val_gen = ImageDataGenerator(rescale=1./255)

train_flow = train_gen.flow_from_dataframe(
    train_data, directory=TRAIN_IMG_DIR,
    x_col='filename', y_col='label',
    target_size=IMG_SIZE, class_mode='binary',
    batch_size=BATCH_SIZE, seed=100
)

val_flow = val_gen.flow_from_dataframe(
    val_data, directory=TRAIN_IMG_DIR,
    x_col='filename', y_col='label',
    target_size=IMG_SIZE, class_mode='binary',
    batch_size=BATCH_SIZE, shuffle=False
)

base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base.trainable = False

x = base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base.input, outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Found 7000 validated image filenames belonging to 2 classes.


Found 3000 validated image filenames belonging to 2 classes.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 75, 75,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 75, 75,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 75, 75,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 75, 75,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 75, 75,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 75, 75,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 75, 75,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 75, 75,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 75, 75,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 75, 75,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 75, 75,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 77, 77,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 38, 38,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 38, 38,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 38, 38,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 38, 38,    │      2,304 │ block_1_depthwis

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
print("Training MobileNetV2 head...")
history = model.fit(
    train_flow,
    epochs=EPOCHS,
    validation_data=val_flow,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)],
    verbose=1
)

Training MobileNetV2 head...
Epoch 1/5


  1/219 ━━━━━━━━━━━━━━━━━━━━ 10:36 3s/step - accuracy: 0.6250 - loss: 0.6783

  2/219 ━━━━━━━━━━━━━━━━━━━━ 13s 64ms/step - accuracy: 0.5625 - loss: 0.7533

  3/219 ━━━━━━━━━━━━━━━━━━━━ 15s 70ms/step - accuracy: 0.5417 - loss: 0.7768

  4/219 ━━━━━━━━━━━━━━━━━━━━ 15s 72ms/step - accuracy: 0.5371 - loss: 0.7796

...

216/219 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9447 - loss: 0.1312

217/219 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9447 - loss: 0.1312

218/219 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9447 - loss: 0.1312

219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.9447 - loss: 0.1311

219/219 ━━━━━━━━━━━━━━━━━━━━ 23s 104ms/step - accuracy: 0.9464 - loss: 0.1279 - val_accuracy: 0.9580 - val_loss: 0.1060


### Validation Performance

In [7]:
val_flow.reset()
preds = model.predict(val_flow)
pred_classes = (preds > 0.5).astype(int).flatten()
true_classes = val_flow.classes
class_labels = list(val_flow.class_indices.keys())

cnn_acc = accuracy_score(true_classes, pred_classes)
print(f"MobileNetV2 Accuracy: {cnn_acc:.4f}")
print(classification_report(true_classes, pred_classes, target_names=class_labels))

cm = confusion_matrix(true_classes, pred_classes)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix -- MobileNetV2')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig('cv_cm_v2.png', dpi=150)
plt.show()

 1/94 ━━━━━━━━━━━━━━━━━━━━ 59s 644ms/step

 2/94 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step  

 3/94 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step

 4/94 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step

 5/94 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step

...

92/94 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step

93/94 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step

94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step


MobileNetV2 Accuracy: 0.9580
              precision    recall  f1-score   support

         cat       0.96      0.96      0.96      1497
         dog       0.96      0.96      0.96      1503

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000



In [8]:
print(f"\n=== Model Comparison ===")
print(f"Random Forest (colour hist): {rf_acc:.4f}")
print(f"MobileNetV2 (transfer):      {cnn_acc:.4f}")


=== Model Comparison ===
Random Forest (colour hist): 0.6367
MobileNetV2 (transfer):      0.9580


MobileNetV2 significantly outperforms the classical baseline, validating the advantage of pretrained deep features for fine-grained visual recognition.

## 5 -- Test Predictions

In [9]:
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))
test_df['filename'] = test_df['id'].astype(str) + '.jpg'

test_flow = val_gen.flow_from_dataframe(
    test_df, directory=TEST_IMG_DIR,
    x_col='filename', y_col=None,
    target_size=IMG_SIZE, class_mode=None,
    batch_size=BATCH_SIZE, shuffle=False
)

print("Predicting on 1,000 test images...")
test_preds = model.predict(test_flow)

idx_map = val_flow.class_indices
rev_map = {v: k for k, v in idx_map.items()}
labels = [rev_map[1] if p > 0.5 else rev_map[0] for p in test_preds]

test_df['prediction'] = labels
test_df[['id', 'prediction']].to_csv('test-predictions_v2.csv', index=False)
print(f"Saved 1,000 predictions to test-predictions_v2.csv")
test_df[['id', 'prediction']].head()

Found 1000 validated image filenames.


Predicting on 1,000 test images...


 1/32 ━━━━━━━━━━━━━━━━━━━━ 19s 639ms/step

 2/32 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step  

 3/32 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step

...

31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step

32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step


Saved 1,000 predictions to test-predictions_v2.csv


,id,prediction
0,1,dog
1,2,cat
2,3,dog
3,4,cat
4,5,dog
